In [58]:
import pandas as pd
import os
from google.cloud import bigquery
import seaborn as sns
import numpy as np
import pickle
from joblib import dump, load
import warnings
from geopy.distance import great_circle

In [49]:
taxi = pd.read_pickle("/Users/liyannie/Desktop/Sp_2019_courses/DS_Indus/project/liyan/df1.pkl")
final_test = pd.read_csv("/Users/liyannie/Desktop/Sp_2019_courses/DS_Indus/project/APM4990_final_test_data_filtered.csv")
weather15 = pd.read_pickle("/Users/liyannie/Desktop/Sp_2019_courses/DS_Indus/project/weather15.pkl")

# Query weather 2015 data

In [2]:
pd.set_option('display.max_columns', 500)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/liyannie/Desktop/Sp_2019_courses/DS_Indus/project/My_First_Project-1cc8e6bacdcb.json"

In [3]:
# get 2015 weather data for final_test
query1 = (
    """
    SELECT concat(year,'-',mo,'-',da) as date_of_year, mo,da,temp,visib,wdsp,gust,max,min,prcp,fog,rain_drizzle,snow_ice_pellets
    FROM `bigquery-public-data.noaa_gsod.gsod2015` where stn='725053'
    """
)

In [5]:
weather15=pd.read_gbq(query1,dialect='standard')

In [6]:
weather15.to_pickle("/Users/liyannie/Desktop/Sp_2019_courses/DS_Indus/project/weather15.pkl")

# Combine with final test set

In [50]:
final_test['pickup_datetime'] = pd.to_datetime(final_test['pickup_datetime'])

In [51]:
final_test.loc[:, 'date_of_year'] = final_test['pickup_datetime'].dt.date
final_test.loc[:, 'hour_of_day'] = final_test['pickup_datetime'].dt.hour
final_test.loc[:, 'day_of_week'] = final_test['pickup_datetime'].dt.dayofweek

In [52]:
final_test['date_of_year'] = final_test['date_of_year'].astype(str)

In [53]:
final_test = pd.merge(final_test, weather15, on='date_of_year')

In [54]:
final_test.shape

(694047, 21)

In [55]:
final_test.columns

Index(['pickup_datetime', 'pickup_latitude', 'pickup_longitude',
       'dropoff_latitude', 'dropoff_longitude', 'passenger_count',
       'date_of_year', 'hour_of_day', 'day_of_week', 'mo', 'da', 'temp',
       'visib', 'wdsp', 'gust', 'max', 'min', 'prcp', 'fog', 'rain_drizzle',
       'snow_ice_pellets'],
      dtype='object')

# Adding features

## 1. Distance

In [56]:
def distance_gc(row):
    try:
        coords_1 = (row['pickup_latitude'], row['pickup_longitude'])
        coords_2 = (row['dropoff_latitude'], row['dropoff_longitude'])
        return great_circle(coords_1, coords_2).miles
    except:
        return None

In [57]:
def distance_manhattan(row):
    try:
        coords_1 = (row['pickup_latitude'], row['pickup_longitude'])
        coords_2 = (row['pickup_latitude'], row['dropoff_longitude'])
        coords_3 = (row['dropoff_latitude'], row['dropoff_longitude'])
        return great_circle(coords_1, coords_2).miles + great_circle(coords_2, coords_3).miles
    except:
        return None

In [59]:
final_test['haver_dist'] = final_test.apply(distance_gc, axis=1)

In [61]:
final_test['man_dist'] = final_test.apply(distance_manhattan, axis=1)

## 2. day of week and hour_of_day

In [63]:
final_test.loc[:, 'hour_of_day'] = final_test['pickup_datetime'].dt.hour
final_test.loc[:, 'day_of_week'] = final_test['pickup_datetime'].dt.dayofweek

## 3. work (1 or 0) and weekday (1 or 0)

In [64]:
final_test['work'] = np.where( (final_test['hour_of_day'] > 7) & (final_test['hour_of_day'] < 19), 1, 0)
final_test['weekday'] = np.where( (final_test['day_of_week'] != 5) & (final_test['day_of_week'] != 6), 1, 0)

## 4. Clustering

In [65]:
# Center lat and lon between pickup and dropoff
final_test.loc[:, 'center_latitude'] = (final_test['pickup_latitude'].values + final_test['dropoff_latitude'].values) / 2
final_test.loc[:, 'center_longitude'] = (final_test['pickup_longitude'].values + final_test['dropoff_longitude'].values) / 2

In [66]:
kmeans = load('kmeans.joblib') 

In [67]:
warnings.filterwarnings('ignore')

pickup_coords_te = final_test[['pickup_latitude', 'pickup_longitude']]
final_test['pickup_cluster'] = kmeans.predict(pickup_coords_te)

dropoff_coords_te = final_test[['dropoff_latitude', 'dropoff_longitude']]
final_test['dropoff_cluster'] = kmeans.predict(dropoff_coords_te)

center_coords_te = final_test[['center_latitude', 'center_longitude']]
final_test['center_cluster'] = kmeans.predict(center_coords_te)

## 5. Avg num of rides in each 3 clusters, for that week day, at that hour. 

In [68]:
def num_rides(df):
    
    num_rides_pcluster = df.groupby(['pickup_cluster', 'day_of_week', 'hour_of_day']).count()[['pickup_latitude']].rename(columns={'pickup_latitude':'num_rides_pcluster'})
    num_rides_pcluster['num_rides_pcluster'] = num_rides_pcluster['num_rides_pcluster'].apply(lambda x: x/19)
    df = pd.merge(df, num_rides_pcluster, on=['pickup_cluster','day_of_week', 'hour_of_day'])

    num_rides_dcluster = df.groupby(['dropoff_cluster', 'day_of_week', 'hour_of_day']).count()[['dropoff_latitude']].rename(columns={'dropoff_latitude':'num_rides_dcluster'})
    num_rides_dcluster['num_rides_dcluster'] = num_rides_dcluster['num_rides_dcluster'].apply(lambda x: x/19)
    df = pd.merge(df, num_rides_dcluster, on=['dropoff_cluster','day_of_week', 'hour_of_day'])

    num_rides_ccluster = df.groupby(['center_cluster', 'day_of_week', 'hour_of_day']).count()[['center_latitude']].rename(columns={'center_latitude':'num_rides_ccluster'})
    num_rides_ccluster['num_rides_ccluster'] = num_rides_ccluster['num_rides_ccluster'].apply(lambda x: x/19)
    df = pd.merge(df, num_rides_ccluster, on=['center_cluster','day_of_week', 'hour_of_day'])
    
    return df

In [69]:
final_test = num_rides(final_test)

In [70]:
final_test.columns

Index(['pickup_datetime', 'pickup_latitude', 'pickup_longitude',
       'dropoff_latitude', 'dropoff_longitude', 'passenger_count',
       'date_of_year', 'hour_of_day', 'day_of_week', 'mo', 'da', 'temp',
       'visib', 'wdsp', 'gust', 'max', 'min', 'prcp', 'fog', 'rain_drizzle',
       'snow_ice_pellets', 'haver_dist', 'man_dist', 'work', 'weekday',
       'center_latitude', 'center_longitude', 'pickup_cluster',
       'dropoff_cluster', 'center_cluster', 'num_rides_pcluster',
       'num_rides_dcluster', 'num_rides_ccluster'],
      dtype='object')

# Removing columns that won't be used for prediction

In [71]:
final_test = final_test.drop(labels = ['pickup_datetime', 'date_of_year', 'da', 
                              'center_latitude', 'center_longitude', 
                              'center_cluster'], 
                    axis = 1)

# Make predictions

In [72]:
# load best trained model
xg_grid2 = load('xg_grid2.joblib') # xg + oHE + target

In [73]:
# make predictions
my_predictions = xg_grid2.predict(final_test)

In [75]:
final_test['predictions'] = my_predictions

In [76]:
final_test.head()

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,hour_of_day,day_of_week,mo,temp,visib,wdsp,gust,max,min,prcp,fog,rain_drizzle,snow_ice_pellets,haver_dist,man_dist,work,weekday,pickup_cluster,dropoff_cluster,num_rides_pcluster,num_rides_dcluster,num_rides_ccluster,predictions
0,40.702740,-74.011650,40.773361,-73.990311,5,21,3,06,66.8,9.8,3.3,999.9,71.1,64.0,0.00,0,0,0,5.005735,5.997238,0,1,20,76,6.368421,1.842105,6.263158,454.061584
1,40.704395,-74.009605,40.775311,-73.984108,1,21,3,01,34.4,8.3,3.4,999.9,39.9,25.0,0.00,0,0,0,5.078338,6.235296,0,1,64,76,9.894737,1.842105,6.263158,559.447083
2,40.706589,-74.014160,40.767136,-73.984451,1,21,3,07,76.7,10.0,4.1,14.0,86.0,68.0,0.00,0,0,0,4.463169,5.739449,0,1,20,72,6.368421,2.052632,6.263158,420.923584
3,40.707321,-74.011978,40.765381,-73.982422,1,21,3,10,59.8,9.8,7.6,21.0,79.0,55.9,0.04,0,1,0,4.299633,5.559592,0,1,20,72,6.368421,2.052632,6.263158,510.088226
4,40.702961,-74.011124,40.764774,-73.981094,1,21,3,03,45.1,10.0,7.7,25.1,59.0,39.9,0.01,0,0,0,4.551083,5.843824,0,1,64,72,9.894737,2.052632,6.263158,565.720520


In [77]:
final_test.to_csv('/Users/liyannie/Desktop/Sp_2019_courses/DS_Indus/project/liyan/my_predictions.csv')